# Modules

In [1]:
%pylab --no-import-all inline
#import EKPSolvers
import gurobipy as grb

Populating the interactive namespace from numpy and matplotlib


# Initialization

In [35]:
c = np.array([0.1,0.3,0.2,0.5])
w = np.array([1,1,3,2])
rhs = 5

c_dict = { i : c[i] for i in range(0,len(c)) }
w_dict = { i : w[i] for i in range(0,len(w)) }

vtype = grb.GRB.BINARY

# Optimization

In [36]:
model = grb.Model("knapsack")
vars = model.addVars(range(0,len(c_dict)),lb=0.0,ub=1.0, \
                     obj=c_dict,vtype=vtype)
model.addConstr(vars.prod(w_dict),grb.GRB.EQUAL,rhs)

fname = 'test.lp'
model.write(fname)

with open(fname) as f:
    content = ''
    for line in f.readlines():
        content = content + line
print(content)



\ Model knapsack
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  0.1 C0 + 0.3 C1 + 0.2 C2 + 0.5 C3
Subject To
 R0: C0 + C1 + 3 C2 + 2 C3 = 5
Bounds
Binaries
 C0 C1 C2 C3
End



In [37]:
model.optimize()

Optimize a model with 1 rows, 4 columns and 4 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 5e+00]
Found heuristic solution: objective 0.6000000
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0.6 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e-01, best bound 6.000000000000e-01, gap 0.0000%


# Solution

In [39]:
x = np.array([v.X for v in vars.values()])

print('objective:','{:.20f}'.format(sum(x*c)))
print('constraint:',sum(x*w),'=',rhs)
print('solution:',x)

objective: 0.60000000000000008882
constraint: 5.0 = 5
solution: [1. 1. 1. 0.]
